In [8]:
from IPython.display import Image

## Generate dag and test out PCalg

![Dag](img/dag-pearl.png)

In [6]:
import numpy as np 
import pandas as pd
SAMPLE_SIZE = 10000

z1 = np.random.normal(0, 1, SAMPLE_SIZE)
z2 = np.random.uniform(0, 1, SAMPLE_SIZE)
z3 = z1*z2 + np.random.normal(0, 1, SAMPLE_SIZE)

w1 = z1**2 + np.random.normal(0, 1, SAMPLE_SIZE)
w2 = 3 * z2 + np.random.standard_cauchy(SAMPLE_SIZE)

x = w1 * z3 + np.random.lognormal(0, 1, SAMPLE_SIZE)
w3 = x + x**3 + np.random.normal(0, 1, SAMPLE_SIZE)
y = w3 * z3 + w2 + np.random.normal(0, 1, SAMPLE_SIZE)


data = {'z1': z1, 'z2': z2, 'z3': z3,
       'w1': w1, 'w2': w2, 'w3': w3,
        'x': x, 'y': y}
